# 朴素贝叶斯算法

机器学习中的逻辑回归、决策树等模型属于**判别方法**，即直接学习标签$y$与特征$X$之间的关系，得到决策函数$y = f(X)$或条件分布$P(Y|X)$；但朴素贝叶斯数据**生成方法**，它直接找标签与特征的联合分布$P(X, Y)$，然后通过
$$
P(Y|X) = \dfrac{P(X, Y)}{P(X)}
$$
计算得出判定结果。

其中各部分含义为

- $P(Y|X)$：后验概率，已知样本特征，样本属于类别$Y$的概率（最终计算目标）；

- $P(Y)$：先验概率，在不考虑任何特征的情况下，样本$Y$出现的占比；

- $P(X|Y)$：似然概率，已知类别$Y$的前提下，样本出现特征$X$的概率；

**朴素贝叶斯的核心假设——给定类别$Y$的条件下，样本的各特征之间独立，即**
$$
P(X|Y) = P(X_1|Y)\cdots P(X_2|Y) \cdots P(X_n|Y)
$$

- $P(X)$：特征$X$本身出现的概率，固定值，用于归一化；


朴素贝叶斯算法的核心思想是通过考虑特征概率来预测分类，即对于给出的待分类样本，求解在此样本出现的条件下各个类别出现的概率，哪个最大，就认为此待分类样本属于哪个类别。

设$X=\{a_1,a_2,\cdots,a_n\}$为一个待分类项，每个$a_i$为$x$的一个特征属性，且特征属性之间相互独立。设$C=\{y_1, y_2, \cdots,y_n\}$为一个类别集合，计算
$$
P(y_k|X) = \text{max}\{ P(y_1|X), P(y_2|X), \cdots, P(y_n|X) \}
$$

$$
P(y_k|X) = \dfrac{P(X|y_i)P(y_i)}{P(X)}
$$
分母是固定的常数，只需考虑分子
$$
\begin{aligned}
P(X|y_i)P(y_i) &= P(a_1|y_i)P(a_2|y_2)\cdots P(a_n|y_i)P(y_i)\\
&= P(y_i)\prod_{j=1}^n{P(a_j|y_i)}\\
\end{aligned}
$$
其中$n$是一个样本的特征数，可以推出
$$
P(X|y_i) = \prod_{j=1}^n{P(a_j|y_i)}
$$


In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split


iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df["label"] = iris.target
df.columns = [
    "sepal length", "sepal width", "petal length", "petal width", "label"
]

data = np.array(df.iloc[:100, :])
X = data[:, :-1]
y = data[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [3]:
class NaiveBayes:
    def __init__(self):
        self.model = None

    @staticmethod
    def mean(X):
        return sum(X) / float(len(X))

    def stdev(self, X):
        avg = self.mean(X)
        return math.sqrt(sum([pow(x - avg, 2) for x in X]) / float(len(X)))

    def gaussian_probability(self, x, mean, stdev):
        exponent = math.exp(-(math.pow(x - mean, 2) /
                              (2 * math.pow(stdev, 2))))
        return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent

    def summarize(self, train_data):
        summaries = [(self.mean(i), self.stdev(i)) for i in zip(*train_data)]
        return summaries

    def fit(self, X, y):
        labels = list(set(y))
        data = {label: [] for label in labels}
        for f, label in zip(X, y):
            data[label].append(f)
        self.model = {
            label: self.summarize(value)
            for label, value in data.items()
        }
        return "GaussianNB Train"

    def calculate_probabilities(self, input_data):
        probabilities = {}
        for label, value in self.model.items():
            probabilities[label] = 1
            for i in range(len(value)):
                mean, stdev = value[i]
                probabilities[label] *= self.gaussian_probability(
                    input_data[i], mean, stdev)
        return probabilities

    def predict(self, X_test):
        label = sorted(
            self.calculate_probabilities(X_test).items(),
            key=lambda x: x[-1])[-1][0]
        return label

    def score(self, X_test, y_test):
        right = 0
        for X, y in zip(X_test, y_test):
            label = self.predict(X)
            if label == y:
                right += 1

        return right / float(len(X_test))

In [4]:
model = NaiveBayes()

In [5]:
model.fit(X_train, y_train)

'GaussianNB Train'

In [6]:
print(model.predict([4.4,  3.2,  1.3,  0.2]))

0.0


## scikit-learn

In [7]:
from sklearn.naive_bayes import GaussianNB, BernoulliNB

In [8]:
clf = GaussianNB()
# clf = BernoulliNB()
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

In [9]:
clf.predict([[4.4, 3.2, 1.3, 0.2]])

array([0.])